# Chapter 02 Object-Oriented Programming

## 2.3 Class Definitions
In python, every data is an instance of some class.

### Example: Credit Card Class

In [54]:
print(list().__class__.__name__)
print(str().__class__.__name__)
print(dict().__class__.__name__)

list
str
dict


In [55]:
print(list().__class__.__class__.__name__)

type


The `self` identifier explicitly identifies the instance that a method is invoked in class.


In [8]:
class CreditCard:
    """A consumer credit card"""
    
    def __init__(self, customer, bank, acnt, limit):
        """Create a new credit card instance.
        
        The initial balance is zero.
        
        :param customer: the name of the customer 
        :param bank: the name of the bank
        :param acnt: the account identifier
        :param limit: credit limit
        """
        
        self._customer = customer
        self._bank = bank
        self._account = acnt
        self._limit = limit
        self._balance = 0
        
    def get_customer(self):
        """Return name of the customer"""
        return self._customer
    
    def get_bank(self):
        """Return the bank's name."""
        return self._bank
    
    def get_account(self):
        """Return the card identifying number (typically stored as a string."""
        return self._account
    
    def get_limit(self):
        """Return current credit limit."""
        return self._limit
    
    def get_balance(self):
        """Return current balance."""
        return self._balance
    
    def charge(self, price):
        """Charge given price to the card, assuming sufficient credit limit.
        
        Return True if charge was processed; False if charge was denied.
        """
        
        if price + self._balance > self._limit:
            return False
        else:
            self._balance += price
            return True
    
    def make_payment(self, amount):
        """Process customer payment that reduces balance."""
        self._balance -= amount 

Python interpreter automatically binds the instance upon which the method is invoked to the `self` parameter.


In [9]:
cc = CreditCard('John Doe', 'Bank', '1234 5678', 1000)
cc

In class, `__init__` method works as the **constructor** of the class. Also, single leading underscore in the name of a data member, such as `_balance`, implies that it is intended as **nonpublic**. Users of a class should not directly access such members.

For better encapsulation, it is mostly better to treat all data members as nonpublic and provide accessors, to provide a user of our class read-only access to a trait, and update methods for updating its memebers.

### Operator overloading and Python's Special Methods
By default, operators can not work on classes unless special methods are defined. You can see detailed info at [here](https://docs.python.org/3/reference/datamodel.html#special-method-names). Python also supports non-operator overloads. For example, `str` invokes `__str__()` and `bool` invokes `__bool__()`. However, we should not carelessly assume that Python will manage all the implications. Defining `__eq__` will support syntax `a == b`, but it does not affect the evaluation of a syntax `a != b`, which should be defined by `__ne__`. In similar context, defining `__eq__` and `__lt__` does not imply semantics for a `a <= b `.

### Example: Multidimensional Vector Class

In [10]:
class Vector:
    """Represent a vector in a multidimensional space"""
    
    def __init__(self, d):
        """Create d-dimensional vector of zeros."""
        self._coords = [0] * d
        
    def __len__(self):
        """Return the dimension of the vector."""
        return len(self._coords)
    
    def __getitem__(self, j):
        """Return jth coordinate of vector."""
        return self._coords[j]
    
    def __setitem__(self, j, val):
        """Set jth coordinate of vector to given value."""
        self._coords[j] = val
    
    def __add__(self, other):
        """Return sum of two vectors."""
        if len(self) != len(other):
            raise ValueError('dimensions must agree')
        result = Vector(len(self))
        for j in range(len(self)):
            result[j] = self[j] + other[j]
        return result
    
    def __eq__(self, other):
        """Return True if vectgor has same coordinates as other."""
        return self._coords == other._coords
    
    def __ne__(self, other):
        """Return True if vector differs from other."""
        return not self == other  # This rely on existing __eq__ definition
    
    def __str__(self):
        """Produce string representation of vector."""
        return '<' + str(self._coords)[1:-1] + '>'
    
    def __repr__(self):
        """For representation."""
        return '<' + str(self._coords)[1:-1] + '>'

In [11]:
x = Vector(5)
x[2] = 10
x

<0, 0, 10, 0, 0>

In [12]:
x + x

<0, 0, 20, 0, 0>

In [13]:
x * x  # Gives error since it is not defined through `__mul__`

TypeError: unsupported operand type(s) for *: 'Vector' and 'Vector'

### Iterators
An **Iterator** for a collection provides one key behavior: It supports a special method named `__next__` that returns the next element of the collection, if any, or raises a `StopIteration` exception to indicate that there are no further elements.

Python automatically supports iterator implementation if both `__len__` and `__getitem__` is defined.

In [14]:
class SequenceIterator:
    """An iterator for any of Python's sequence types."""
    
    def __init__(self, sequence):
        """Create an iterator for the given sequence."""
        self._seq = sequence  # keep a reference to the underlying data
        self._k = -1  # will increment to 0 on first call to next
    
    def __next__(self):
        """Return the next element, or else raise StopIteration error."""
        self._k += 1
        if self._k < len(self._seq):
            return(self._seq[self._k])
        else:
            raise StopIteration()
        
    def __iter__(self):
        """By convention, an iterator must return itself as an iterator."""
        return self

In [15]:
custom_iter = SequenceIterator([4, 2, 1])
iter(custom_iter)

In [16]:
for i in SequenceIterator([5, 2, 1, 2, 7]):
    print(i)

5
2
1
2
7


### Example: Range Class
This example mimics Pthon's built-in `range` class. There is a big difference between Python 3's `range` and Python 2's range. Basically, Python 3's `range` works like a Python 2's `xrange`.

The core difference between them is that Python 3's range employs **lazy evaluation**, which does not create a new list instance, which might be expensive if the list is huge, effectively repesenting the desired range of elements without storing them in memory.

In [17]:
class Range:
    """A class that mimic's the built-in range class."""
    
    def __init__(self, start, stop=None, step=1):
        """Initialize a Range Instance
        Semantics is similar to built-in range class
        """
        if step == 0:
            raise ValueError('step cannot be 0')
        
        if stop is None:
            start, stop = 0, start
            
        # calculate the effective length once
        self._length = max(0, (stop - start + step -1) // step)
        
        # nned knowledge of start and step (but not step) to support _-getitem__
        self._start = start
        self._step = step
        
    def __len__(self):
        """Return number of entries in the range."""
        return self._length
    
    def __getitem__(self, k):
        """Return entry at index k (using standard interpretation if negative.)"""
        if k < 0:
            k += len(self)
        
        if not 0 <= k < self._length:
            raise IndexError('index out of range')
        
        return self._start + k * self._step

## 2.4 Inheritance
In object-oriented programming, the mechanism for a modular and hierarchical organization is a technique known as **inheritance**. This allows a new class to be defined based upon an existing class as the starting point. In object-oriented terminology, the existing class is typically described as the **base class**, **parent class**, or **superclass**, while the newly defined classs is known as the **subclass** or **child class**.

### Extending the CreditCard Class

In [18]:
class PredatoryCreditCard(CreditCard):
    """An extension to CreditCard that compounds interest and fees."""
    
    def __init__(self, customer, bank, acnt, limit, apr):
        """Create a new predatory credit card instance.
        
        The initial balance is zero.
        customer    the name of the customer
        bank        the name of the bank
        acnt        the acount identifier
        limit       credit limit
        apr         annual percentage rate
        """

        super().__init__(customer, bank, acnt, limit)
        self._apr = apr
        
    def charge(self, price):
        """Charge given price to the card, assuming sufficient credit limit.
        
        Return True if charge was processed.
        Return False and assess $5 fee if charge is denied.
        """
        
        success = super().charge(price)
        if not success:
            self._balance += 5
        return success
    
    def process_month(self):
        """Assess monthly interest on outstanding balance."""
        if self._balance > 0:
            # if positive balance, convert APR to monthly multiplicative factor
            monthly_factor = pow(1 + self._apr, 1/12)
            self._balance *= monthly_factor

In [19]:
predatory = PredatoryCreditCard('John', 'AA Bank', '1234 5678', 100000000, 0.03)
predatory.charge(1000)
predatory.get_balance()

1000

In [22]:
predatory.process_month()
predatory.get_balance()

1007.4170717777332

#### Protected Members
In Python, names beginning with a single underscore are conventionally akin to **protected**, which are accessible to subclasses, but not to the general public, while names beginning with a double underscore (other than special methods) are akin to **private**, which are not accessible to both subclasses and general public.

However, this access control is not supported formally by Python. Keep in mind that it is just a convention. So if you are trying to use some attributes followed by an underscore, think twice that it could compromise the class's designer's intention.

### Hierarchy of Numeric Progressions
As a second example of use of inheritance, we develop a hierarchy of classes for iterating numeric progressions.

![Hierarchy of Progression Classes](../images/Fig2.7.png)

In [35]:
class Progression:
    """ITerator producing a generic progression.
    
    Default iterator produces the whole numbers 0, 1, 2, ...
    """
    
    def __init__(self, start=0):
        """Initialize current to the first value of the progression."""
        self._current = start
    
    def _advance(self):
        """Update self._current to a new value.
        
        THis should be overridden by a subclass to customize progression.
        
        By convention, if current is set to None, this designates the end of a progression.
        """
        
        self._current +=1
        
    def __next__(self):
        """Return the next element, or else raise StopIteration error."""
        if self._current is None:
            raise StopIteration()
        else:
            answer = self._current
            self._advance()
            return answer
        
    def __iter__(self):
        """By convention, an iterator must return itself as an iterator."""
        return self
    
    def print_progression(self, n):
        """Print next n values of the progression."""
        print(' '.join(str(next(self)) for j in range(n)))

#### An Arithmetic Progression Class

In [36]:
class ArithmeticProgression(Progression):
    """Iterator producing an arithmetic progression."""
    
    def __init__(self, increment=1, start=0):
        """Create a new arithmetic progression.
        
        increment   the fixed constant to add to each term (default 1)
        start       the first term of the progression (default 0)
        """
        super().__init__(start)
        self._increment = increment
    
    def _advance(self):
        """Update current value by adding the fixed increment."""
        self._current += self._increment

#### A Geometric Progression Class

In [37]:
class GeometricProgression(Progression):
    """Iterator producing a geometric progression."""
    
    def __init__(self, base=2, start=1):
        """Create a new geometric progression.
        
        base    the fixed constant multiplied to each term (default 2)
        start   the first term of the progression (default 1)
        """
        
        super().__init__(start)
        self._base = base
    
    def _advance(self):
        """Update current value by multiplying it by the base value."""
        self._current *= self._base

#### A Fibonacci Progression Class

In [38]:
class FibonacciProgression(Progression):
    """Iterator producing a generalized Fibonacci progression."""
    
    def __init__(self, first=0, second=1):
        """Create a new fibonacci progression.
        
        first       the first term of the progression (default 0)
        second      the second term of the progression (default 1)
        """
        
        super().__init__(first)
        self._prev = second - first
    
    def _advance(self):
        """Update current value by taking sum of previous two."""
        self._prev, self._current = self._current, self._prev + self._current

#### Testing progressions

In [40]:
Progression().print_progression(10)
ArithmeticProgression(5).print_progression(10)
ArithmeticProgression(5, 2).print_progression(10)
GeometricProgression().print_progression(10)
GeometricProgression(3).print_progression(10)
FibonacciProgression().print_progression(10)
FibonacciProgression(4, 6).print_progression(10)

0 1 2 3 4 5 6 7 8 9
0 5 10 15 20 25 30 35 40 45
2 7 12 17 22 27 32 37 42 47
1 2 4 8 16 32 64 128 256 512
1 3 9 27 81 243 729 2187 6561 19683
0 1 1 2 3 5 8 13 21 34
4 6 10 16 26 42 68 110 178 288


### Abstract Base Class
In classic object-oriented terminology, we say a class is an **abstract base class** if its only purpose is to serve as a base class though inheritance. More formally, an abstract base class is one that cannot be directly instantiated, while **concrete class** is one that can be instantiated. By this definition, our `Progression` class is technically concrete, although we essentially designated it as an abstract base class.

In statically typed languages such as Java and C++, an abstract base class serves as a formal type that may guarantee one or more **abstract methods**. This provides support for polymorphism, as a variable may have an abstract base class as its declared type, even though it refers to an instance of a concrete subclass. Because there are no declared types in Python, this kind of polymorphism can be accomplished without the need for a unifying abstract class. For this reason, there is not as strong a tradition of defining abstract base classes in Python, although Python's `abc` module provides support for defining a formal abstract base class.

Our reason for focusing on abstract base classes in our study of data structures is that Python's `collections` module provides several abstract base classes that assist when defining custom data structures that share a common interface with some of Python's built-in data structures. THese rely on an object-oriented software design pattern known as the **template method pattern** The template method pattern is when an abstract base class provides concrete behaviors that rely upon calls to other abstract behaviours. In that way, as soon as a sublclass provides definitions for the missing abstract behaviors, the inherited concrete behaviors are well defined.

Asa tangible example, the `collections.Sequence` abstract base class defines behaviors common to Python's `list`, `str`, and `tuple` classes, as sequences that support element access via an integer index. More so, the `collections.Sequence` class provides concrete implementations of methods, `count`, `index` and `__contains__` that can be inherited by any class that provides concrete implementations of both `__len__` and `__getitem__`.

In [41]:
from abc import ABCMeta, abstractmethod


class Sequence(metaclass=ABCMeta):
    """Our own version of collections.Sequence abstract base class."""
    
    @abstractmethod
    def __len__(self):
        """Return the length of the sequence."""
        
    @abstractmethod
    def __getitem__(self, item):
        """Return the element at index item of the sequence."""
    
    def __contains__(self, val):
        """Retrun True if val found in the sequence; False otherwise."""
        for j in range(len(self)):
            if self[j] == val:
                return True
        return False

    def index(self, val):
        """REturn leftmost index at which val is found (or raise ValueError)."""
        for j in range(len(self)):
            if self[j] == val:
                return j
        raise ValueError('value not in sequence')
    
    def count(self, val):
        """Return the number of elements equal to given value."""
        k = 0
        for j in range(len(self)):
            if self[j] == val:
                k += 1
            return k

In [44]:
Sequence()

TypeError: Can't instantiate abstract class Sequence with abstract methods __getitem__, __len__

As we can see, since `Sequence` is an abstract Class, not an concrete class, so that it cannot be instantiated.

This implementation relies on two advanced Python techniques. THe first is that we declare `ABCMeta` class of the `abc` module as a **metaclass** of our `Sequence` class. *A meta class is different from a superclass*, in that it provides a template for the class definition itself. Specifically, the `ABCMeta` declaration assures that the constructor for the class raises an error.

The second advanced technique is the use of the `@abstractmethod` decorator immediately before the `__len__` and `__getitem__` methods are declared. That declares these two particular methods to be abstract, meaning that we do not provide an implementation within our Sequence base class, but that we expect any concrete subclasses to support those two methods. Python enforces this expectation, by disallowing instantitation for any subclass that does not override the abstract methods with concrete implementations.

We emphasize that if a subclass provides its own implementation of an inherited behaviors from a base class, the new definition overrides the inherited one. THis technique can be used when we have the ability to provide a more efficient implementation for a behavior than is achieved by the generic approach. 

## 2.5 Namespaces and Object-Orientation
A **namespace** is an abstraction that manages all of the identifiers that are defined in a particular scope, mapping each name to its associated value. In Python, functions, classes, and modules are all first-class objects, and so the "value" associated with an identifier in a namespace may in fact be a function, class, or module.


### Instance and Class namespaces
We begin by exploring what is known as the **instance namespace**, which manages attributes specific to an individual object. For example, each instance of our `CreditCard` class maintains a distinct balance, a distinct account number, a distinct credit limit, and so on. Each credit card will have a dedicated instance namespace to manage such values.

There is a separate **class namespace** for each class that has been defined. This namespace is used to manage members that are to be *shared* by all instances of a class, or used without reference to any particular instance. For example, `make_payment` method of the `CreditCard` class is not stored independently by each instance of that class. That member function is stored within the namespace of the `CreditCard` class. Our `Predatory CreditCard` class has its own namespace, containing the three methods we defined for that subclass: `__init__`, `charge` and `process_month`.

#### How Entries Are Established in a Namespace
When inheritance is used, there is still a single *instance namespace* per object.

A *class namespace* includes all declarations that are made directly within the body of the class definition.

#### Class Data Members
A class-level data member is often used when there is some value, such as a constant, that is to be shared by all instances of a class. In such a case, it would be unnecessarily wasteful to have each instance store that value in its instance namespace.

```python
class PredatoryCreditCard(CreditCard):
    OVERLIMIT_FEE = 5    # this is a class-level member
    
    def charge(self, price):
        success = super().charge(price)
        if not success:
            self._balance += PredatoryCreditCard.OVERLIMIT_FEE
        return success
```

The data member, `OVERLIMIT_FEE`, is entered into the `PredatoryCreditCard` class namespace because that assignment takes place within the immediate scope of the class definition, and without any qualifying identifier.

#### Nested Classes
It is also possble to nest one class definition within the scope of another class. This is a useful construct, which we wil exploit several times in this book in the implementation of data structures. This can be done by using a syntax such as

```python
class A:
    class B:
```

In this case, class `B` is the nested class. The identifier `B` is entered in to the namespace of class `A` associated with the newly defined class. We note that this technique is unrelated to the concept of inheritance, as class `B` does not inherit from class `A`.

Nesting one class in the scope of another makes clear that the nested class exists for support of the outer class. Furthermore, it can help reduce potential name conflicts, because it allows for a similarly named class to exist in another context. For example we will later introduce a data structure known as a **linked list** and will define a nested node class to store the individual components of the list. We will also introduce a data sturcture known as a **tree** that depends upon its own nested node class. Thes e two structures rely on different node definitions, and by nesting those within the respective container classes, we avoid ambiguity.

Another advantage of one class being nested as a member of another is taht it allows for a more advanced form of inheritance in which a subclass of the outer class overrides the definition of its nested class.

#### Dictionaries and the `__slots__` Declaration
By default, Python represents each namespace with an instance of the built-in `dict` class that maps identifying names in that scope to the associated objects. While a dictionary structure supports relatively efficient anme lookups, it requires additional memory usage beyond the raw data that it stores.

Python provides a more direct mechnism for representing instance namespaces that avoids the use of an auxiliary dictionary. To use the streamlined representation for all instances of a class, that class definition must provide a class-levle member named `__slots__` that is assigned to a fixed sequene of strings that serve as names fro instance variables. For example, with our `CreditCard` class, we would declare the following:

```python
class CreditCard:
    __slots__ = '_customer', '_bank', '_account', '_balance', '_limit'
```

In this example, the righ-hand side of the assignment is technically a tuple.

When inheritance is used, if the base class declares `__slots__`, a subclass must also declare `__slots__` to avoid creation of instance dictionaries. The declaration in the subclass should only include names of supplemental methods that are newly introduced. For example, our `PredatoryCreditCard` declaration would include the following declaration:

```python
class PredatoryCreditCard(CreditCard):
    __slots__ = '_apr'  # in addition to the inherited members
```

#### Name Resolution and Dynamic Dispatch
In this section, we examine the process that is used when *retrieving* a name in Python's object-oriented framework. When the dot operator syantax is used to access an existing number, such as `obj.foo`, the Python interpreter begins a name resolution process, described as follows:

1. The instance namespace is searched; if the desired name is found, its associated value is used.
2. Otherwise the class namespace, for the class to which the instance belongs, is searched; if the name is found, its associated value is used.
3. If the name was not found in the immediate class namespace, the search continues upward through the inheritance hierarchy, checking the class namespace for each ancestor (commonly checking the superclass, then its superclass and so on). The first time the name is found, its associate value is used.
4. If the name has still not found, an `AttributeError` is raised.

In traditional object-oriented terminology, Python uses what is known as **dynamic dispatch** (or **dynamic binding**) to determine, at run-time, which iplementation of a function to call based upon the type of the object upon which it is invoked. This is contrast to some languages that use **static dispatching**, making a compile-time decision as to which version of a function to call, based upon the declared type of a variable.

## 2.6 Shallow and Deep Copying
We emphasized that an assignment statement `foo = bar` makes the name `foo` an **alias** for the object identified as `bar`. In this section, we consider the task of making an **copy** of an object, rather than alias. This is necessary in applications when we want to subsequently modify either the original or the copy in an independent manner.

Consider a scenario in which we manage various lists of colors, with each color represented by an instance of a presumed `color` class. We let identifier `warmtones` denote an existing list of such colors. In this application, we wish to create a new list named `palette`, which is a copy of `warmtones` list. However, we want to subsequently be able to add additional colors to `palette`, or to modify or remove some of the existing colors, without affecting the contents of `warmwtones`. If we were to exectue the command

```python
palette = warmtones
```

this creates an alias. No new list is created; instead the new identifier `palette` references the original list.

Unfortunately, this does not meet our desired criteria, because if we subsequently add or remove colors from `palette`, we modify the list identified as `warmtones`.

We can instead create a new instance of the `list` class by using the syntax:

```python
palette = list(warmtones)
``` 

In this case, we explicitly call the `list` constructor, sending the first list as a parameter. This cause a new list to be created; however it is what is known as a **shallow copy**. The  new list is initialized so that its contents are precisely the same as the original sequence. However, Python's list are **referential**, and so the new list represents a sequnce of references to the same elements as in the first.

This is a better situation than our first attempt, as we can legitimately add or remove elements from the `palette` without affecting `warmtones`. However, if we edit a `color` instance from the `palette` list, we effectively change the contents of `wamtones`. Although `palette` and `wamtones` are distinct list, there remains indirect aliasing, for example, with `palette[0]` and `warmtones[0]` as aliases for the same color instance.

We prefer that `palette` be what is known as a **deep copy** of `warmtones`. In a deep copy, the new copy references its own *copies* of those objects referenced by the original version.

#### Python's copy Module
To create a deep copy, we could populate our list by explicitly making copies of the original instances, but this requires that we know how to make copies of colors (rather than aliasing). Python provides a very convenient module, named `copy`, that can produce both shallow copies and deep copies of arbitrary objects.

This module supports two functions: the `copy` function create a shallow copy of its argument, and the `deepcopy` function creates a deep copy of its argument. After importing the module, we may create a deep copy for our example, using the command:

```python
palette = copy.deepcopy(warmtones)
```